# Preparation

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 404 error pages

In [ ]:
col_name = ['URL', 'Title', 'Pageviews']

In [ ]:
df_404 = pd.read_csv('./drive/My Drive/Colab Notebooks/01_dataset/error/1.csv', names=col_name)

In [ ]:
df_404 = df_404.drop(index=[0,1,2,3,4,5])

In [ ]:
df_404 = df_404.drop(['Title'], axis=1)

In [ ]:
df_404 = df_404.rename(columns={'Pageviews': 'Pageviews(20200801-20200930)'})

In [ ]:
len(df_404['URL'].unique())

In [ ]:
df_404 = df_404.reset_index(drop=True)

In [ ]:
df_404_products = df_404[df_404['URL'].str.contains('/products/')]

In [ ]:
df_404_products = df_404_products.reset_index(drop=True)

In [ ]:
df_404_products['URL'].str.split('/products/', expand=True)

In [ ]:
df_404_products['Product_ID'] = df_404_products['URL'].str.split('/products/', expand=True)[1].str.split('?', expand=True)[0]

In [ ]:
len(df_404_products['Product_ID'].unique())

In [ ]:
df_404_products = df_404_products.drop_duplicates(subset='Product_ID')

In [ ]:
df_404_products = df_404_products.reset_index(drop=True)

In [ ]:
df_404_products['URL'] = 'https://www.decathlon.co.jp' + df_404_products['URL'].astype(str)

In [ ]:
df_404_products.to_csv('./drive/My Drive/Colab Notebooks/01_dataset/all_404_error_products_20200801-20200930.csv')

# All page views for 30 days from 1st to 30th April 2020

In [ ]:
df_old_arr = []

In [ ]:
for i in range(1,26):
    file = pd.read_csv('./drive/My Drive/Colab Notebooks/01_dataset/old/Analytics Main View all unique page views 20200401-20200430-{}.csv'.format(i), names=col_name)
    file = file.drop(index=[0,1,2,3,4,5])
    df_old_arr.append(file)

In [ ]:
df_old = pd.concat(df_old_arr, ignore_index=True)

In [ ]:
df_old = df_old.rename(columns={'Pageviews': 'Pageviews(20200401-20200430)'})

In [ ]:
df_old_products = df_old[df_old['URL'].str.contains('/products/')]

In [ ]:
df_old_products['Product_ID'] = df_old_products['URL'].str.split('/products/', expand=True)[1].str.split('?', expand=True)[0]

In [ ]:
df_old_products = df_old_products.drop_duplicates(subset='Product_ID')

In [ ]:
df_old_products['URL'] = 'https://www.decathlon.co.jp' + df_old_products['URL'].astype(str)

In [ ]:
df_old_products = df_old_products.reset_index(drop=True)

In [ ]:
df_old_products.loc[0:, 'URL']

In [ ]:
df_old_products.to_csv('./drive/My Drive/Colab Notebooks/01_dataset/all_pageviews_products_20200401-20200430.csv')

# All page views for 14 days from 17th to 30th September 2020

In [ ]:
df_new_arr = []

In [ ]:
for i in range(1,18):
    file = pd.read_csv('./drive/My Drive/Colab Notebooks/01_dataset/new/Analytics Main View all unique page views 20200917-20200930-{}.csv'.format(i), names=col_name)
    file = file.drop(index=[0,1,2,3,4,5])
    df_new_arr.append(file)

In [ ]:
df_new = pd.concat(df_new_arr, ignore_index=True)

In [ ]:
df_new = df_new.rename(columns={'Pageviews': 'Pageviews(20200917-20200930)'})

In [ ]:
df_new.to_csv('./drive/My Drive/Colab Notebooks/01_dataset/all_pageviews_20200917-20200930.csv')

# Merge Dataframes

## 1st merge with 404 error and old page views

In [ ]:
oldUrlAndTitle = pd.merge(df_404_products, df_old_products, on='Product_ID')

### Pages which are still getting 404 error even now

In [ ]:
still_404 = oldUrlAndTitle[oldUrlAndTitle['Title'] == '404 Not Found | デカトロン日本公式オンラインストア']

In [ ]:
still_404['URL'] = 'https://www.decathlon.co.jp' + still_404['URL'].astype(str)

In [ ]:
still_404 = still_404[['URL_y', 'Pageviews(20200401-20200430)', 'URL_x', 'Pageviews(20200801-20200930)', 'Title', 'Product_ID']]

In [ ]:
still_404 = still_404.reset_index(drop=True)

In [ ]:
len(still_404['Product_ID'].unique())

In [ ]:
still_404.to_csv('./drive/My Drive/Colab Notebooks/01_dataset/updated_still_404.csv')

### 1st Merged dataframe

In [ ]:
# rows with unique values
len(oldUrlAndTitle['Title'].unique())

In [ ]:
# rows with 404 error
len(oldUrlAndTitle[oldUrlAndTitle['Title'] == '404 Not Found | デカトロン日本公式オンラインストア'])

In [ ]:
# remove rows with 404 error page title
oldUrlAndTitle = oldUrlAndTitle[oldUrlAndTitle['Title'] != '404 Not Found | デカトロン日本公式オンラインストア']

In [ ]:
# change the URL column name
oldUrlAndTitle = oldUrlAndTitle.rename(columns={'URL_x': 'old_URL_x', 'URL_y': 'old_URL_y'})

In [ ]:
oldUrlAndTitle = oldUrlAndTitle[['old_URL_y', 'Pageviews(20200401-20200430)', 'old_URL_x', 'Pageviews(20200801-20200930)', 'Title', 'Product_ID']]

## 2nd merge with the above-merged one and new page views

In [ ]:
oldUrlAndNewUrlAndTitle = pd.merge(oldUrlAndTitle, df_new, on='Title')

In [ ]:
# change the URL column name
oldUrlAndNewUrlAndTitle = oldUrlAndNewUrlAndTitle.rename(columns={'URL': 'new_URL', 'Product_ID': 'old_Product_ID'})

In [ ]:
oldUrlAndNewUrlAndTitle = oldUrlAndNewUrlAndTitle.drop(['Pageviews(20200401-20200430)', 'old_URL_y'], axis=1)

In [ ]:
# change the URL column name
oldUrlAndNewUrlAndTitle = oldUrlAndNewUrlAndTitle.rename(columns={'old_URL_x': 'old_URL'})

In [ ]:
oldUrlAndNewUrlAndTitle = oldUrlAndNewUrlAndTitle[['old_URL', 'Pageviews(20200801-20200930)', 'old_Product_ID', 'Title', 'new_URL', 'Pageviews(20200917-20200930)']]

In [ ]:
len(oldUrlAndNewUrlAndTitle['Title'].unique())

In [ ]:
for i in range(20):
    print(oldUrlAndNewUrlAndTitle['old_URL'][i])

In [ ]:
for i in range(10):
    print(oldUrlAndNewUrlAndTitle['Title'][i])

In [ ]:
for i in range(20):
    print(oldUrlAndNewUrlAndTitle['new_URL'][i])

In [ ]:
# remove rows with duplicate ones in Title
oldUrlAndNewUrlAndTitle = oldUrlAndNewUrlAndTitle.drop_duplicates(subset=['Title'])

In [ ]:
# reset index number
oldUrlAndNewUrlAndTitle = oldUrlAndNewUrlAndTitle.reset_index(drop=True)

In [ ]:
oldUrlAndNewUrlAndTitle['new_URL'].str.split('/products/', expand=True)[1].str.split('?', expand=True)[0]

In [ ]:
oldUrlAndNewUrlAndTitle['new_Product_ID'] = oldUrlAndNewUrlAndTitle['new_URL'].str.split('/products/', expand=True)[1].str.split('?', expand=True)[0]

In [ ]:
oldUrlAndNewUrlAndTitle['new_URL'] = 'https://www.decathlon.co.jp' + oldUrlAndNewUrlAndTitle['new_URL'].astype(str)

In [ ]:
oldUrlAndNewUrlAndTitle.to_csv('./drive/My Drive/Colab Notebooks/01_dataset/updated_oldAndNewUrls.csv')